In [68]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import (
    Descriptors, rdMolDescriptors, AllChem 
    )
import networkx as nx

In [69]:
df_cleaned_antibody = pd.read_csv('../data/df_cleaned_antibody.csv')
df_cleaned_antibody_copy = df_cleaned_antibody.copy()
df_cleaned_antibody_copy.tail()

,SMILES,bacteria,MIC (µg/mL),MIC (µmol/mL),log MIC,S. aureus,molar_mass
927,N(=N/C1C(=O)N(N=C1C)C2=CC(Cl)=CC=C2)\C3=CC=C(C...,Staphylococcus aureus ATCC 9144,125.00,0.291455,2.464571,+,428.883
928,N(=N/C1C(=O)N(N=C1C)C2=CC(F)=CC=C2)\C3=CC=C(C=...,Staphylococcus aureus ATCC 9144,15.62,0.037873,1.578333,+,412.428
929,FC1=CC=C(C=C1)C2N(N=C(C=3C=CC(=CC3)NCN4C=NC=5C...,Staphylococcus aureus ATCC 9144,15.62,0.033843,1.529468,+,461.544
930,FC(F)(F)C1=CC=C(C=C1)C2N(N=C(C=3C=CC(=CC3)NCN4...,Staphylococcus aureus ATCC 9144,15.62,0.030535,1.484792,+,511.551
931,O=C(N1N=C(C=C1C)C)CN2C=3C=CC=CC3N=C2C4CC(=O)N(...,Staphylococcus aureus ATCC 9144,250.00,0.579413,2.762988,+,431.471


In [70]:
df_cleaned_antibody_copy.shape

(932, 7)

In [71]:
def generate_basic_descriptors(smiles_str: str) -> dict:
    """
    Генерирует основные молекулярные дескрипторы из SMILES строки.
    
    Возвращает словарь с дескрипторами.
    """
    mol = Chem.MolFromSmiles(smiles_str)
    
    descriptors = {
        'mol_MR': Descriptors.MolMR(mol), # Молекулярный рефрактивный индекс
        'logP': Descriptors.MolLogP(mol), # Липофильность (коэффициент распределения LogP)
        'num_h_donors': Descriptors.NumHDonors(mol), # Число доноров водородных связей
        'num_h_acceptors': Descriptors.NumHAcceptors(mol), # Число акцепторов водородных связей
        'num_rotatable_bonds': Descriptors.NumRotatableBonds(mol), # Число вращаемых связей
        'tpsa': Descriptors.TPSA(mol), # Топологическая площадь полярной поверхности (TPSA)
        'fraction_csp3': rdMolDescriptors.CalcFractionCSP3(mol), # Доля атомов углерода в sp3-гибридизации
        'ring_count': mol.GetRingInfo().NumRings(), # Число колец в молекуле
        'aromatic_ring_count': rdMolDescriptors.CalcNumAromaticRings(mol) # Число ароматических колец
    }
    
    return descriptors


def add_descriptors_to_dataset(df: pd.DataFrame, smiles_column: str) -> pd.DataFrame:
    """
    Добавляет дескрипторы для каждого соединения на основе SMILES строки.
    
    Параметры:
    df: pandas DataFrame - исходный датасет.
    smiles_column: str - название столбца с SMILES.

    Возвращает:
    pandas DataFrame - датасет с новыми колонками дескрипторов.
    """
    descriptors_list = []

    for smiles in df[smiles_column]:
        descriptors = generate_basic_descriptors(smiles)
        descriptors_list.append(descriptors)

    descriptors_df = pd.DataFrame(descriptors_list)

    df_with_descriptors = pd.concat([df, descriptors_df], axis=1)

    return df_with_descriptors


def smiles_to_graph(smiles_str: str) -> nx.Graph:
    """
    Преобразует SMILES в граф с использованием RDKit и NetworkX.
    Вершины — это атомы, рёбра — это связи.
    
    Параметры:
    smiles_str: str - строка SMILES, представляющая молекулу.
    
    Возвращает:
    graph: nx.Graph - граф молекулы, где атомы являются вершинами, а связи рёбрами.
    """
    
    mol = Chem.MolFromSmiles(smiles_str)
    
    # Создаем пустой граф с использованием NetworkX
    graph = nx.Graph()
    
    # Добавляем вершины (атомы)
    for atom in mol.GetAtoms():
        graph.add_node(atom.GetIdx(), element=atom.GetSymbol())
    
    # Добавляем рёбра (связи)
    for bond in mol.GetBonds():
        graph.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx(), bond_type=bond.GetBondType())
    
    return graph


def generate_graph_metrics_from_graph(graph: nx.Graph) -> dict:
    """
    Генерирует графовые метрики для уже созданного графа: PageRank, Degree Centrality,
    Closeness Centrality и Betweenness Centrality.

    Возвращает словарь с агрегированными метриками для всей молекулы.
    """

    # PageRank: показывает вес (важность) атомов в структуре
    pagerank = nx.pagerank(graph)

    # Degree Centrality: показывает, сколько связей имеет каждый атом
    degree_centrality = nx.degree_centrality(graph)

    # Closeness Centrality: указывает, насколько атом близок к другим атомам в молекуле
    closeness_centrality = nx.closeness_centrality(graph)

    # Betweenness Centrality: измеряет, насколько часто атом лежит на кратчайших путях между другими атомами
    betweenness_centrality = nx.betweenness_centrality(graph)

    # Агрегация
    metrics = {
        'mean_pagerank': sum(pagerank.values()) / len(pagerank),                   # Среднее значение PageRank по всей молекуле
        'mean_degree_centrality': sum(degree_centrality.values()) / len(degree_centrality),  # Среднее значение Degree Centrality по всей молекуле
        'mean_closeness_centrality': sum(closeness_centrality.values()) / len(closeness_centrality),  # Среднее значение Closeness Centrality
        'mean_betweenness_centrality': sum(betweenness_centrality.values()) / len(betweenness_centrality)  # Среднее значение Betweenness Centrality
    }

    return metrics


def add_graph_metrics_to_dataset(df: pd.DataFrame, smiles_column: str) -> pd.DataFrame:
    """
    Добавляет графовые метрики (PageRank, Degree Centrality, Closeness Centrality, Betweenness Centrality)
    к каждому SMILES в датасете.
    """
    graph_metrics_list = []

    for smiles in df[smiles_column]:
        graph = smiles_to_graph(smiles)
        metrics = generate_graph_metrics_from_graph(graph)
        graph_metrics_list.append(metrics)

    metrics_df = pd.DataFrame(graph_metrics_list)
    
    df_with_metrics = pd.concat([df, metrics_df], axis=1)
    
    return df_with_metrics

In [72]:
df_with_features_rdkit = add_descriptors_to_dataset(df_cleaned_antibody_copy, 'SMILES')

df_with_features_rdkit.head()

,SMILES,bacteria,MIC (µg/mL),MIC (µmol/mL),log MIC,S. aureus,molar_mass,mol_MR,logP,num_h_donors,num_h_acceptors,num_rotatable_bonds,tpsa,fraction_csp3,ring_count,aromatic_ring_count
0,N#CC=1C(=O)N(N=C(C2=NC=3C=CC=CC3N2)C)C(N=CC=4C...,Methicillin-resistant S. aureus (ATCC 43300),50.00,0.089133,1.950037,+,560.961,154.6311,5.71946,1,9,6,166.12,0.034483,5,5
1,N#CC=1C(=O)N(N=C(C2=NC=3C=CC=CC3N2)C)C(N=CC=4C...,Methicillin-resistant S. aureus (ATCC 43300),12.50,0.020647,1.314859,+,605.412,157.3211,5.82856,1,9,6,166.12,0.034483,5,5
2,N#CC=1C(=O)N(N=C(C2=NC=3C=CC=CC3N2)C)C(N=CC4=C...,Methicillin-resistant S. aureus (ATCC 43300),6.25,0.010324,1.013829,+,605.412,157.3211,5.82856,1,9,6,166.12,0.034483,5,5
3,ClC1=CC=2N=C(Cl)N(C2C=C1Cl)C3CCCC3,methicillin-resistant S. aureus (MRSA),6.25,0.021582,1.334092,+,289.593,72.2680,5.11160,0,2,1,17.82,0.416667,3,2
4,ClC1=CC=2N=C(Br)N(C2C=C1Cl)C3CCCC3,methicillin-resistant S. aureus (MRSA),6.25,0.018710,1.272076,+,334.044,74.9580,5.22070,0,2,1,17.82,0.416667,3,2


In [73]:
df_with_rdkit_and_graph_metrics = add_graph_metrics_to_dataset(df_with_features_rdkit, 'SMILES')

df_with_rdkit_and_graph_metrics.to_csv('../data/df_with_features_rdkit_and_graph.csv', index=False)

In [74]:
df_with_rdkit_and_graph_metrics.head()

,SMILES,bacteria,MIC (µg/mL),MIC (µmol/mL),log MIC,S. aureus,molar_mass,mol_MR,logP,num_h_donors,num_h_acceptors,num_rotatable_bonds,tpsa,fraction_csp3,ring_count,aromatic_ring_count,mean_pagerank,mean_degree_centrality,mean_closeness_centrality,mean_betweenness_centrality
0,N#CC=1C(=O)N(N=C(C2=NC=3C=CC=CC3N2)C)C(N=CC=4C...,Methicillin-resistant S. aureus (ATCC 43300),50.00,0.089133,1.950037,+,560.961,154.6311,5.71946,1,9,6,166.12,0.034483,5,5,0.024390,0.054878,0.154891,0.148124
1,N#CC=1C(=O)N(N=C(C2=NC=3C=CC=CC3N2)C)C(N=CC=4C...,Methicillin-resistant S. aureus (ATCC 43300),12.50,0.020647,1.314859,+,605.412,157.3211,5.82856,1,9,6,166.12,0.034483,5,5,0.024390,0.054878,0.154891,0.148124
2,N#CC=1C(=O)N(N=C(C2=NC=3C=CC=CC3N2)C)C(N=CC4=C...,Methicillin-resistant S. aureus (ATCC 43300),6.25,0.010324,1.013829,+,605.412,157.3211,5.82856,1,9,6,166.12,0.034483,5,5,0.024390,0.054878,0.154093,0.149187
3,ClC1=CC=2N=C(Cl)N(C2C=C1Cl)C3CCCC3,methicillin-resistant S. aureus (MRSA),6.25,0.021582,1.334092,+,289.593,72.2680,5.11160,0,2,1,17.82,0.416667,3,2,0.058824,0.139706,0.298309,0.165196
4,ClC1=CC=2N=C(Br)N(C2C=C1Cl)C3CCCC3,methicillin-resistant S. aureus (MRSA),6.25,0.018710,1.272076,+,334.044,74.9580,5.22070,0,2,1,17.82,0.416667,3,2,0.058824,0.139706,0.298309,0.165196
